# Folium
Folium is a tool that makes you look like a mapping God while all the work is done in the back end.

It's a Python wrapper for a tool called leaflet.js. We basically give it minimal instructions, JS does loads of work in the background and we get some very, very cool maps. It's great stuff.

Show a map with a pop up on the city hall

In [27]:
import numpy as np
import pandas as pd
import folium


tiles_available = [
    "OpenStreetMap",
    "Mapbox Bright",
    "Mapbox Control Room",
    "Mapbox Dark",
    "Stamen Terrain",
    "Stamen Toner",
    "Stamen Watercolor",
    "CartoDB dark_matter",
    "CartoDB positron"
]

map = folium.Map(location=[37.77919, -122.41914],
                    tiles = "CartoDB positron",
                    attr="Custom tiles by Stamen Design",
                    zoom_start = 16,
                    #width = '50%',
                    #height = '50%'
                    ) # Uses lat then lon. The bigger the zoom number, the closer in you get

folium.Marker([37.77919, -122.41914], popup='London Bridge').add_to(map)
map 
# Calls the map to display  

## More complex markers

The London Bridge marker is the same as above, I just added a little colour with that extra line of green.

The second marker is more interesting. First, it's not a marker like the pin. The pin is an icon type marker. The circle is essentially just a coloured overlay, so we use a different colour command.

`CircleMarker` radius is set in pixels so if you change the zoom you need to change the pixels. It can also take a `fill_color` that's semi-transparent.

the third marker is an interactive marker
Let your users add marker with the following code:

```python
markers = map_hooray.add_child(folium.ClickForMarker(popup="pop_up_name"))

In [30]:
# Set the map up
map_hooray = folium.Map(location=[51.5074, 0.1278],
                        tiles = "CartoDB positron",
                        attr="Custom tiles by Stamen Design",
                        zoom_start = 9)
# Simple marker
folium.Marker([51.5079, 0.0877],
              popup='London Bridge',
              icon=folium.Icon(color='green')
             ).add_to(map_hooray)

# Circle marker
folium.CircleMarker([51.4183, 0.2206],
                    radius=30,
                    popup='East London',
                    color='red',
                    ).add_to(map_hooray)

# Interactive marker
map_hooray.add_child(folium.ClickForMarker(popup="Dave is awesome"))

map_hooray

In [44]:
data = pd.read_csv('../week1/Police_Department_Incident_Reports__Historical_2003_to_May_2018_20240129.csv') 
data['Date'] = pd.to_datetime(data['Date'])

In [70]:
data1 = data[(data["Category"] == "DRUG/NARCOTIC") & 
             (data["Date"] >= pd.to_datetime('2017-12-01')) & 
             (data["Date"] <= pd.to_datetime('2018-01-01'))]

# Set the map up
map_hooray = folium.Map(location=[37.77919, -122.41914],
                        tiles = "CartoDB positron",
                        attr="Custom tiles by Stamen Design",
                        zoom_start = 12)

# Circle marker
for point in data1["location"].values:
    # Remove "POINT (" from the start and ")" from the end, then split on space
    coords = point[7:-1].split()
    longitude = float(coords[0])
    latitude = float(coords[1])
    
    folium.CircleMarker([latitude, longitude],
                    radius=3,
                    popup='East London',
                    color='red',
                    ).add_to(map_hooray)

map_hooray

# Heatmaps

In [92]:
from folium import plugins
from folium.plugins import HeatMap

# Set the map up
heatmap = folium.Map(location=[37.77919, -122.41914],
                        tiles = "CartoDB positron",
                        attr="Custom tiles by Stamen Design",
                        zoom_start = 12)


data_assault = data[data['Category'] == 'SEX OFFENSES, NON FORCIBLE']
# print(type(data_assault['X'].values[0]))

heat_df = data_assault[['X', 'Y']]
print(heat_df.head())
heat_df = heat_df.dropna(axis=0, subset=['X','Y'])

# # List comprehension to make out list of lists
heat_data = [[row['Y'],row['X']] for index, row in heat_df.iterrows()]
print(heat_data[0:5])

# # Plot it on the map
HeatMap(heat_data).add_to(heatmap)

# # Display the map
heatmap


                 X          Y
69845  -122.444518  37.783421
138370 -122.452609  37.786569
204041 -122.411359  37.776494
312858 -122.417044  37.789595
336083 -122.390923  37.719879
[[37.7834206344523, -122.444518269835], [37.7865686524807, -122.452608514061], [37.7764944808779, -122.411359138357], [37.7895952774322, -122.417044292614], [37.7198785524783, -122.390922553406]]


# Heatmap movie

In [94]:
from folium import plugins

# Set the map up
map_hooray = folium.Map(location=[37.77919, -122.41914],
                        tiles = "CartoDB positron",
                        attr="Custom tiles by Stamen Design",
                        zoom_start = 12)


data_assault = data[data['Category'] == 'ARSON']
# print(type(data_assault['X'].values[0]))

heat_df = data_assault[['X', 'Y']]
print(heat_df.head())
heat_df = heat_df.dropna(axis=0, subset=['X','Y'])

# Convert the 'Date' column to datetime
data_assault['Date'] = pd.to_datetime(data_assault['Date'])

# Group the data by date and count the number of occurrences
daily_counts = data_assault.groupby(data_assault['Date'].dt.date).size()

# Create a list of lists containing the Y, longitude, and weight (number of occurrences) for each date
heat_data = [[row[2], row[1], count] for row, count in zip(heat_df.itertuples(), daily_counts)]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_hooray)
# Display the map
map_hooray

               X          Y
3    -122.436220  37.724377
920  -122.401829  37.735606
1844 -122.391350  37.725667
2373 -122.387182  37.720967
2671 -122.420004  37.782942


C:\Users\Admin\AppData\Local\Temp\ipykernel_32276\3066676031.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_assault['Date'] = pd.to_datetime(data_assault['Date'])
